# LEAST COST PATH GENERATION SCRIPT
Allison Smith
June 03, 2023

This script reads in slope, vegetation density, and terrain roughness layers generated from 3DEP lidar data, preprocesses data, creates rate and pace rasters, and generates least cost paths using ArcGIS tools.

## Load packages and data

In [ ]:
print("Importing libraries...")
import os
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("spatial")

print(ctime() + " setting up directories, processing extent...")
main_dir = "E:\\a_smith\\lakeMtnsSite\\"
slope_dir = "E:\\a_smith\\lakeMtnsSite\\slope\\slope.tif"
veg_dir = "E:\\a_smith\\lakeMtnsSite\\veg\\veg.tif"
terr_dir = "E:\\a_smith\\lakeMtnsSite\\terrain\\roughness.tif"
dtm_dir = "E:\\a_smith\\lakeMtnsSite\\dtm\\dtm.tif"

print(ctime() + " loading data...")
slope = Raster(slope_dir)
veg = Raster(veg_dir)
terr = Raster(terr_dir)
dtm = Raster(dtm_dir)
studySite = r"E:\\a_smith\\lakeMtnsSite\\lakemtns\\initial_lcp\\studySite.shp"
start1 = r"E:\\a_smith\\lakeMtnsSite\\lakemtns\\initial_lcp\\start1.shp"
start2 = r"E:\\a_smith\\lakeMtnsSite\\lakemtns\\initial_lcp\\start2.shp"
start3 = r"E:\\a_smith\\lakeMtnsSite\\lakemtns\\initial_lcp\\start3.shp"
safetyZone = r"E:\\a_smith\\lakeMtnsSite\\lakemtns\\initial_lcp\\safetyZone.shp"

working_dir = main_dir + "lcp_generation\\"
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
env.workspace = working_dir

## Preprocessing layers

In this step we are trimming the slope, vegetation density, and terrain roughness layers down to the study site size and aggregating them to the correct resolution. 

In [ ]:
print(ctime() + " trimming to study site...")
slo_sub = ExtractByMask(slope, studySite, "INSIDE", studySite)
veg_sub = ExtractByMask(veg, studySite, "INSIDE", studySite)
terr_sub = ExtractByMask(terr, studySite, "INSIDE", studySite)
dtm_sub = ExtractByMask(dtm, studySite, "INSIDE", studySite)

In [ ]:
print(ctime() + " aggregating to 20m cell size...")
slo_agg = Aggregate(slo_sub, 20, "MEAN", "EXPAND", "NODATA")
#veg does not get aggregated because it is the limiting cell size (20m)
#veg_agg = Aggregate(veg_sub, cell_factor, "MEAN", "EXPAND", "NODATA")
terr_agg = Aggregate(terr_sub, 4, "MEAN", "EXPAND", "NODATA")
dtm_agg = Aggregate(dtm_sub, 20, "MEAN", "EXPAND", "NODATA")

## Raster Calculations

Here we are combining the slope, vegetation, and terrain layers to generate the travel rate and pace rasters for least cost path generation. 

In [ ]:
#the travel rate calculation is subject to change as the new equation becomes available
print(ctime() + " performing raster calculations...")
travel_rt = RasterCalculator ([slo_agg, veg_sub, terr_agg], [s, v, t], 
                              (1.662 + (-1.076 * v) + (-9.011 * t) + (-0.05191 * s) + (-0.01127 * (s*s))),
                              "IntersectionOf", "MaxOf")
travel_rt.save("travelRt.tif")
travel_pc = RasterCalculator ([travel_rt], [tr], (1/tr))
travel_pc.save("travelPc.tif")

## LCP Operations

In the final step, we are creating the distance accumulation rasters and back direction rasters which feed the optimal path as line tool to generate our LCPs for each starting point. 

In [ ]:
#saving out back direction rasters
outback1 = "outback1.tif"
outback2 = "outback2.tif"
outback3 = "outback3.tif"

In [ ]:
print(ctime() + " creating distance accumulation rasters...")
dist_acc1 = DistanceAccumulation(in_source_data = start1, 
                                 in_surface_raster = dtm_agg, 
                                 in_cost_raster = travel_pc, 
                                 out_back_direction_raster = outback1)
dist_acc2 = DistanceAccumulation(in_source_data = start2, 
                                 in_surface_raster = dtm_agg, 
                                 in_cost_raster = travel_pc, 
                                 out_back_direction_raster = outback2)
dist_acc3 = DistanceAccumulation(in_source_data = start3, 
                                 in_surface_raster = dtm_agg, 
                                 in_cost_raster = travel_pc, 
                                 out_back_direction_raster = outback3)

In [ ]:
print(ctime() + " generating LCPs...")
lcp1 = OptimalPathAsLine(in_destination_data = safetyZone, 
                         in_distance_accumulation_raster = dist_acc1, 
                         in_back_direction_raster = outback1, 
                         out_polyline_features = "/lcp1.shp",
                         path_type = "EACH_ZONE")
lcp2 = OptimalPathAsLine(in_destination_data = safetyZone, 
                         in_distance_accumulation_raster = dist_acc2, 
                         in_back_direction_raster = outback2, 
                         out_polyline_features = "/lcp2.shp",
                         path_type = "EACH_ZONE")
lcp3 = OptimalPathAsLine(in_destination_data = safetyZone, 
                         in_distance_accumulation_raster = dist_acc3, 
                         in_back_direction_raster = outback3, 
                         out_polyline_features = "/lcp3.shp",
                         path_type = "EACH_ZONE")